In [1]:
from optimizers.ConstraintsOpt import SFW 
from optimizers.ConstraintsSet import create_lp_constraints, create_k_sparse_constraints, make_feasible
from model import SimpleNet
from serialization import ndarrays_to_sparse_parameters, sparse_parameters_to_ndarrays
from torchvision.models import resnet18
import torch
import numpy as np

2024-04-25 14:30:53.652258: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 14:30:53.652292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 14:30:53.652948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 14:30:53.657000: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 14:30:54.643508: W tensorflow/compiler/tf2

In [2]:
model = SimpleNet()
resnet = resnet18(pretrained=False, num_classes=10)

/home/tunguyen/.conda/envs/flower-simulation/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tunguyen/.conda/envs/flower-simulation/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
constraints = create_lp_constraints(resnet, ord=1, mode='radius')
make_feasible(resnet, constraints)

In [5]:
params = {}
for name, p in resnet.named_parameters():
    np_p = p.detach().numpy()
    try:
        sparse_p = ndarrays_to_sparse_parameters(np_p)
        print(f"Successfully converted {name} : {np_p.shape}")
    except RuntimeError as e:
        print(f"Failed to convert {name} to sparse")
        print("RuntimeError: ", e)
        print("np_p: ", np_p.shape)
        continue
    params[name] = sparse_p

/home/tunguyen/federated-learning/simulation-flwr/serialization.py:30: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /opt/conda/conda-bld/pytorch_1699449183005/work/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  tensor = torch.tensor(ndarray).to_sparse_csr()


Successfully converted conv1.weight : (64, 3, 7, 7)
Successfully converted bn1.weight : (64,)
Successfully converted bn1.bias : (64,)
Successfully converted layer1.0.conv1.weight : (64, 64, 3, 3)
Successfully converted layer1.0.bn1.weight : (64,)
Successfully converted layer1.0.bn1.bias : (64,)
Successfully converted layer1.0.conv2.weight : (64, 64, 3, 3)
Successfully converted layer1.0.bn2.weight : (64,)
Successfully converted layer1.0.bn2.bias : (64,)
Successfully converted layer1.1.conv1.weight : (64, 64, 3, 3)
Successfully converted layer1.1.bn1.weight : (64,)
Successfully converted layer1.1.bn1.bias : (64,)
Successfully converted layer1.1.conv2.weight : (64, 64, 3, 3)
Successfully converted layer1.1.bn2.weight : (64,)
Successfully converted layer1.1.bn2.bias : (64,)
Successfully converted layer2.0.conv1.weight : (128, 64, 3, 3)
Successfully converted layer2.0.bn1.weight : (128,)
Successfully converted layer2.0.bn1.bias : (128,)
Successfully converted layer2.0.conv2.weight : (128, 

In [6]:
params.keys()

dict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer2.0.conv1.weight', 'layer2.0.bn1.weight', 'layer2.0.bn1.bias', 'layer2.0.conv2.weight', 'layer2.0.bn2.weight', 'layer2.0.bn2.bias', 'layer2.0.downsample.0.weight', 'layer2.0.downsample.1.weight', 'layer2.0.downsample.1.bias', 'layer2.1.conv1.weight', 'layer2.1.bn1.weight', 'layer2.1.bn1.bias', 'layer2.1.conv2.weight', 'layer2.1.bn2.weight', 'layer2.1.bn2.bias', 'layer3.0.conv1.weight', 'layer3.0.bn1.weight', 'layer3.0.bn1.bias', 'layer3.0.conv2.weight', 'layer3.0.bn2.weight', 'layer3.0.bn2.bias', 'layer3.0.downsample.0.weight', 'layer3.0.downsample.1.weight', 'layer3.0.downsample.1.bias', 'layer3.1.conv1.weight', 'layer3.1.bn1.weight', 'layer3.1.bn1.bias', 

In [7]:
npparams = [val.detach().numpy() for val in resnet.parameters()]

In [14]:
nbarray.shape

(64, 3, 7, 7)

In [12]:
nbarray = npparams[0]
nbarray.reshape((nbarray.shape[0], -1)).shape

(64, 147)

In [12]:
myparams = [np.random.random_sample(val.shape) for val in npparams]

In [20]:
diff_p = []
for mod_pa, ran_pa in zip(npparams, myparams):
    diff = np.maximum(np.abs(mod_pa) - np.abs(ran_pa), 0)
    diff_p.append(diff)

In [21]:
alpha = 1
mask = np.random.choice([True,False], size=nbarray.shape, p=[alpha, 1-alpha])
nbarray[mask] = 0

In [22]:
nbarray

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]],


       [[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0.

In [37]:
layer = 
constraints_lp[layer].lmo(params[layer].data),constraints_ksparse[layer].lmo(params[layer].data)

IndexError: list index out of range

In [ ]:
sfw_optim = SFW(params, constraints_lp, constraints_ksparse, model, lr=0.01, max_iter=1000, tol=1e-5)